In [1]:
with open('api.txt','r') as r:
    TEXT = r.read()

In [2]:
from ecommerce.src_base_scraper.scraper_builder import ScraperManagerBuilder
from ecommerce.src_base_scraper.requests_handler import RequestsHandler
from ecommerce.scraper_data.functions import get_pages,get_product,get_product_data

In [3]:
handler = RequestsHandler.create_instance(None)
builder = ScraperManagerBuilder(requests_handler = handler,
                      initial_url='https://www.dedeman.ro/ro/mobilier-birou/c/96',
                      initial_node_function = get_pages,
                      node_functions=[get_product],
                      product_functions = get_product_data
                      )

In [4]:
scraper = builder.build()

In [5]:
a = scraper.get_data(parallel_processing_flag=True,depth_parallelism=0)

In [6]:
from ecommerce.data_parsing.iterator_parser import EcommerceFuncGenerator

In [17]:
scrfunc = EcommerceFuncGenerator(requests_handler=handler,
                                 first_url = 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page={2}',
                                 max_page = 10
                                 )

In [18]:
func = scrfunc.generate_url_function()

In [19]:
list(func(''))

['https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=1',
 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=2',
 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=3',
 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=4',
 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=5',
 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=6',
 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=7',
 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=8',
 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=9',
 'https://www.drmax.ro/remedii-naturiste/sistemul-imunitar?page=10']